# Loading datasets

In [9]:
import pandas as pd

answers = pd.read_json("simple-legal-questions-pl-main/answers.jl", lines=True)
passages = pd.read_json("simple-legal-questions-pl-main/passages.jl", lines=True)
questions = pd.read_json("simple-legal-questions-pl-main/questions.jl", lines=True)    

passages.head()

,_id,title,text
0,2004_2387_1,Ustawa z dnia 27 sierpnia 2004 r. o ratyfikacj...,Art. 1. Wyraża się zgodę na dokonanie przez Pr...
1,2004_2387_2,Ustawa z dnia 27 sierpnia 2004 r. o ratyfikacj...,Art. 2. Ustawa wchodzi w życie po upływie 14 d...
2,2001_1209_1,Ustawa z dnia 26 lipca 2001 r. o nabywaniu prz...,Art. 1. 1. Osobom fizycznym będącym w dniu 26 ...
3,2001_1209_2,Ustawa z dnia 26 lipca 2001 r. o nabywaniu prz...,"Art. 2. 1. Do osób, o których mowa w art. 1 us..."
4,2001_1209_3,Ustawa z dnia 26 lipca 2001 r. o nabywaniu prz...,"Art. 3. 1. Do osób, które skorzystały z uwłasz..."


In [5]:
from datasets import load_dataset

ds = load_dataset("clarin-pl/poquad")
ds_val = ds["validation"]

README.md:   0%|          | 0.00/317 [00:00<?, ?B/s]

d:\Studia\NLP\.venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adam6\.cache\huggingface\hub\datasets--clarin-pl--poquad. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


poquad.py:   0%|          | 0.00/5.35k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/12.5M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/46187 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5764 [00:00<?, ? examples/s]

# Training model/ using pretrained model

In [11]:
import torch
print(torch.cuda.is_available()) 

True


In [17]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model="apohllo/plt5-base-poquad")

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

d:\Studia\NLP\.venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adam6\.cache\huggingface\hub\models--apohllo--plt5-base-poquad. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/833 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.59M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

Device set to use cuda:0


In [18]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("apohllo/plt5-base-poquad")

# Scoring the model

F1 - scoring for QA model implemented from https://qa.fastforwardlabs.com/no%20answer/null%20threshold/bert/distilbert/exact%20match/f1/robust%20predictions/2020/06/09/Evaluating_BERT_on_SQuAD.html

In [138]:
# these functions are heavily influenced by the HF squad_metrics.py script
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

def get_gold_answers(example):
    """helper function that retrieves all possible true answers from a squad2.0 example"""
    
    gold_answers = [answer["text"] for answer in example.answers if answer["text"]]

    # if gold_answers doesn't exist it's because this is a negative example - 
    # the only correct answer is an empty string
    if not gold_answers:
        gold_answers = [""]
        
    return gold_answers

In [139]:
prediction = pipe(ds_val[0]["question"])
answer = ds_val[0]["answers"]["text"][0]

In [140]:
compute_f1(prediction[0]["generated_text"], answer)

0

Calculating F1-score for the whole dataset

In [ ]:
predictions = []
ds_answers = []
for item in ds_val:
    res = pipe(item["question"])
    prediction.append(res[0]["generated_text"])
    ds_answers.append(item["answers"]["text"][0])

In [42]:
predictions = prediction[1:]

In [46]:
print(ds_val[5]['question'])
print(predictions[5])
print(ds_answers[5])

Kiedy Plater wraz z oddziałem dotarła do Dusiatów?
Kiedy Plat z oddziałem dotarła do Dusiatów
29 marca 1831


In [55]:
from numpy import mean

em_score = mean([(compute_exact_match(prediction_, answer)) for answer, prediction_ in zip(ds_answers,predictions)])
f1_score = mean([(compute_f1(prediction_, answer)) for answer, prediction_ in zip(ds_answers,predictions)])

In [59]:
print("Exact match score based on whole set: ", em_score)
print("F1-Score score based on whole set: ", f1_score)

Exact match score based on whole set:  0.0017349063150589867
F1-Score score based on whole set:  0.03289134652286021


In [62]:
df = ds_val.to_pandas()
df.head()

,id,title,context,question,answers
0,1,Miszna,Pisma rabiniczne – w tym Miszna – stanowią kom...,Czym są pisma rabiniczne?,{'text': ['kompilację poglądów różnych rabinów...
1,2,Miszna,Pisma rabiniczne – w tym Miszna – stanowią kom...,Z ilu komponentów składała się Tora przekazana...,"{'text': ['dwóch'], 'answer_start': [172]}"
2,3,Miszna,Pisma rabiniczne – w tym Miszna – stanowią kom...,W jakich formach występowała Tora przekazana M...,"{'text': ['pisanej, a drugą część w formie ust..."
3,4,Miszna,Pisma rabiniczne – w tym Miszna – stanowią kom...,W jakiej formie przekazana została Miszna?,"{'text': ['ustna'], 'answer_start': [269]}"
4,5,Emilia Plater,Sformowany przez nią oddział partyzancki liczy...,Kto początkowo należał do oddziału stworzonego...,"{'text': ['280 strzelców, kilkuset chłopów kos..."


In [63]:
df["predictions"] = predictions
df["f1-scores"] = [(compute_f1(prediction_, answer)) for answer, prediction_ in zip(ds_answers,predictions)]
df["exact match"] = [(compute_exact_match(prediction_, answer)) for answer, prediction_ in zip(ds_answers,predictions)]

Showing the best answers that model provided

In [66]:
df.sort_values("f1-scores",ascending=False).head(10)

,id,title,context,question,answers,predictions,f1-scores,exact match
1978,1979,Szpak zwyczajny,Podczas żerowania szpaki stosują trzy techniki...,Ile technik zdobywania pożywienia wykorzystują...,"{'text': ['trzy'], 'answer_start': [33]}",trzy,1.0,1
39,40,Inwazja Rosji na Ukrainę (2022),"W 2014 roku Rosja, w odpowiedzi na obalenie pr...",W którym roku Rosja dokonała aneksji Krymu?,"{'text': ['W 2014'], 'answer_start': [0]}",W 2014,1.0,1
757,758,HMS Furious (1916),"14 kwietnia „Furious”, eskortowany przez trzy ...","Ile okrętów eskortowało ""Furious""?","{'text': ['trzy'], 'answer_start': [41]}",trzy,1.0,1
4604,4605,Barszcz Sosnowskiego,Z sekcji Pubescentia w Europie występują trzy ...,Ile gatunków Pubescentia istnieje w Europie?,"{'text': ['trzy'], 'answer_start': [41]}",trzy,1.0,1
4417,4418,Konflikt w Górskim Karabachu (2020),9 października rosyjski prezydent Władimir Put...,Gdzie miały się odbyć rozmowy pokojowe zapowie...,"{'text': ['w Moskwie'], 'answer_start': [110]}",w Moskwie,1.0,1
5439,5440,Grzegorz Ciechowski,W 1990 powrócił do śpiewania z zespołem Republ...,Jaki krążek wydał Ciechowski z zespołem w 1991?,"{'text': ['1991'], 'answer_start': [140]}",1991,1.0,1
2094,2095,Ziemia,Temperatura środka planety może wynosić 4000–7...,Izotop którego z pierwiastków ma najdłuższy cz...,"{'text': ['toru'], 'answer_start': [328]}",toru,1.0,1
3371,3372,Incredible Hulk,"Zak Penn, który pracował już nad scenariuszem ...",Ile wersji scenariusza stworzył Penn?,"{'text': ['trzy'], 'answer_start': [220]}",trzy,1.0,1
655,656,Fenenna kujawska,W kwestii kolejności narodzin potomstwa Ziemom...,Ile córek miała książęca para?,"{'text': ['trzy'], 'answer_start': [370]}",trzy,1.0,1
534,535,Podział administracyjny Rosji,De iure (w świetle prawa międzynarodowego) zaa...,Do jakiego kraju należą obszary Sewastopolu i ...,"{'text': ['Ukrainy'], 'answer_start': [165]}",Ukrainy,1.0,1


Showing the worst answers that model provided

In [67]:
df.sort_values("f1-scores",ascending=True).head(10)

,id,title,context,question,answers,predictions,f1-scores,exact match
0,1,Miszna,Pisma rabiniczne – w tym Miszna – stanowią kom...,Czym są pisma rabiniczne?,{'text': ['kompilację poglądów różnych rabinów...,Pismom rabinicznym,0.0,0
3699,3700,Kodeks Synajski,Bibliście pozwolono przewozić kodeks partiami ...,"Dlaczego po przekazaniu kodeksu carowi, Tische...",{'text': ['ponieważ rząd turecki stwarzał pewn...,tak,0.0,0
3698,3699,Kodeks Synajski,Bibliście pozwolono przewozić kodeks partiami ...,Co Tischendorf proponował Cyrylowi w zamian za...,"{'text': ['poparcie rosyjskiego cara'], 'answe...",tak,0.0,0
3696,3697,Daglezja zielona,Daglezja zielona jest w warunkach naturalnych ...,Ile wynosi spadek wzrostu drzewa zakażonego A....,"{'text': ['do 75%'], 'answer_start': [941]}",1%,0.0,0
3694,3695,Daglezja zielona,Daglezja zielona jest w warunkach naturalnych ...,Jakie są skutki infekcji A. douglasii?,{'text': ['wywiera wpływ na tempo wzrostu drze...,douglasii,0.0,0
3692,3693,Hobbit (trylogia filmowa),Zdjęcia rozpoczęły się 21 marca 2011. Film był...,Jakie kamery wykorzystano przy tej produkcji?,"{'text': ['cyfrowymi RED EPIC'], 'answer_start...",kamerę,0.0,0
3691,3692,Hobbit (trylogia filmowa),Zdjęcia rozpoczęły się 21 marca 2011. Film był...,Czy zastosowana przez reżysera nietypowa metod...,{'text': ['Po pierwszych pokazach roboczej wer...,nie,0.0,0
3690,3691,Hobbit (trylogia filmowa),Zdjęcia rozpoczęły się 21 marca 2011. Film był...,W ilu klatkach na sekundę kręci się zazwyczaj ...,"{'text': ['24'], 'answer_start': [254]}",w milionach,0.0,0
3700,3701,Kodeks Synajski,Bibliście pozwolono przewozić kodeks partiami ...,Kto był sceptycznie nastawiony do kandydatury ...,"{'text': ['patriarcha Jerozolimy'], 'answer_st...",Krzysztof,0.0,0
3689,3690,Hobbit (trylogia filmowa),Zdjęcia rozpoczęły się 21 marca 2011. Film był...,Jaki nietypowy klatkaż zdecydował się zastosow...,"{'text': ['48 klatek na sekundę'], 'answer_sta...",wyrzuty,0.0,0


Running the test on Legal questions

In [68]:
questions.head()

,_id,text
0,1,"Czy żołnierz, który dopuszcza się czynnej napa..."
1,2,Z ilu osób składa się komisja przetargowa?
2,3,Do jakiej wysokości za zobowiązania spółki odp...
3,4,"Kiedy ustala się wartość majątku obrotowego, k..."
4,5,"Jakiej karze podlega armator, który wykonuje r..."


In [69]:
answers.head()

,score,question-id,answer
0,1.0,1,"Tak, podlega karze aresztu wojskowego albo poz..."
1,1.0,2,Komisja przetargowa składa się z co najmniej t...
2,1.0,3,Komandytariusz odpowiada za zobowiązania spółk...
3,1.0,4,Wartość rzeczowych składników majątku obrotowe...
4,1.0,5,Podlega karze pieniężnej do wysokości 1 000 00...


In [130]:
legal_questions = []
legal_true_answers = []
legal_generated_answers = []

In [123]:
threashold = int(len(questions)/10)
threashold

143

In [ ]:
import math

for idx, row in questions.iterrows():
  if idx % threashold == 0:
    print(f"Done {math.ceil(idx/len(questions) * 100)} %")
    
  legal_questions.append(row['text'])
  matching_answers = answers[answers['question-id'] == row["_id"]]['answer'].values.tolist()
  legal_true_answers.append(matching_answers[0] if matching_answers else "brak")
  res = pipe(row["text"])
  legal_generated_answers.append(res[0]['generated_text'])

Done 0 %
Done 10 %
Done 20 %
Done 30 %
Done 40 %
Done 50 %
Done 60 %
Done 70 %
Done 80 %
Done 90 %
Done 100 %


In [132]:
df_legal= pd.DataFrame()

In [133]:
df_legal['question'] = legal_questions
df_legal['true answer'] = legal_true_answers
df_legal["predictions"] = legal_generated_answers
df_legal["f1-scores"] = [(compute_f1(prediction_, answer)) for answer, prediction_ in zip(legal_true_answers,legal_generated_answers)]
df_legal["exact match"] = [(compute_exact_match(prediction_, answer)) for answer, prediction_ in zip(legal_true_answers,legal_generated_answers)]

In [134]:
df_legal.head(5)

,question,true answer,predictions,f1-scores,exact match
0,"Czy żołnierz, który dopuszcza się czynnej napa...","Tak, podlega karze aresztu wojskowego albo poz...",nie,0.000000,0
1,Z ilu osób składa się komisja przetargowa?,Komisja przetargowa składa się z co najmniej t...,Z tylu,0.181818,0
2,Do jakiej wysokości za zobowiązania spółki odp...,Komandytariusz odpowiada za zobowiązania spółk...,do jakiej wysokości,0.250000,0
3,"Kiedy ustala się wartość majątku obrotowego, k...",Wartość rzeczowych składników majątku obrotowe...,Kiedy ustala się wartość majątku obrotowego,0.303030,0
4,"Jakiej karze podlega armator, który wykonuje r...",Podlega karze pieniężnej do wysokości 1 000 00...,podlega karze pozbawienia wolności,0.307692,0


In [137]:
print("Exact match score based on whole set: ", df_legal['exact match'].mean())
print("F1-Score score based on whole set: ", df_legal['f1-scores'].mean())

Exact match score based on whole set:  0.034122562674094706
F1-Score score based on whole set:  0.07144071652797317


Showing best answers

In [144]:
df_legal.sort_values('f1-scores',ascending=False).head(10)

,question,true answer,predictions,f1-scores,exact match
705,Czy dyrektorem żłobka może być osoba posiadają...,Nie,nie,1.0,1
1157,Czy za użytkowanie wieczyste gruntów przez zak...,Nie.,nie,1.0,1
700,Czy zażalenie wstrzymuje wykonanie zaskarżoneg...,Nie,nie,1.0,1
1425,Czy producentowi dopiero rozpoczynającemu prod...,nie,nie,1.0,1
1121,Czy dyrektor regionalnej dyrekcji Lasów Państw...,Nie,nie,1.0,1
697,Czy specjalne zezwolenie połowowe zawiera wyks...,Nie,nie,1.0,1
1116,Czy skazani nie mogą kierować skarg do organów...,Nie,nie,1.0,1
1370,Czy na rozprawie mogą być obecne osoby w stani...,nie,nie,1.0,1
1371,Czy urzędnik służby cywilnej ponosi opłaty z t...,nie,nie,1.0,1
696,Czy specjalne zezwolenie połowowe zawiera wyks...,Nie,nie,1.0,1


Showing worst answers

In [147]:
filtered_df = df_legal[df_legal['true answer'] != 'brak']
filtered_df.sort_values('f1-scores',ascending=True).head(10)

,question,true answer,predictions,f1-scores,exact match
0,"Czy żołnierz, który dopuszcza się czynnej napa...","Tak, podlega karze aresztu wojskowego albo poz...",nie,0.0,0
1042,Kto wyznacza termin usunięcia braków wynikłych...,,spółka,0.0,0
1043,Kto wyznacza termin usunięcia braków wynikłych...,sąd rejestrowy,spółka,0.0,0
1044,Kto wyznacza termin usunięcia braków wynikłych...,sąd rejestrowy,spółka,0.0,0
1045,Który spośród kilku ratujących ma prawo do wyn...,,jeden,0.0,0
1046,Który spośród kilku ratujących ma prawo do wyn...,każdy z ratujących,jeden,0.0,0
1047,Który spośród kilku ratujących ma prawo do wyn...,,jeden,0.0,0
1048,Który spośród kilku ratujących ma prawo do wyn...,"Każdy ratujący, chyba że z własnej winy przycz...",jeden,0.0,0
1041,Kto wyznacza termin usunięcia braków wynikłych...,,spółka,0.0,0
1049,Którzy pracownicy Najwyższej Izby Kontroli nie...,,pracownicy Najwyższej Izby Kontroli,0.0,0


# Answers to the questions

#### Does the performance on the validation dataset reflects the performance on your test set?

No, the performance on the validation set was significantly worse than those on legal question set. This is probably because the answers on the validation set are more extensive than legal questions. As we can see on the top chart of the legal questions all answers are single words 'nie'

#### What are the outcomes of the model on your test questions? Are they satisfying? If not, what might be the reason for that?

About the legal questions we can see that model predicts correctly for one word answer let's see what other answers got f1-score 1 in legal dataset

In [ ]:
df_legal[df_legal['f1-scores'] == 1]['predictions'].unique()

array(['nie', 'tak', 'jeden'], dtype=object)

As we can see the perfect scores get only answers consiting one word now let's see the answrs where f1-score is higher than 0.5

In [152]:
df_legal[(df_legal['f1-scores'] > 0.5) & (df_legal['f1-scores'] < 1.0)]['predictions'].unique()

array(['Rada Krajowego Związku Kas', 'Kodeks morski',
       'podlega karze pozbawienia wolności', 'na wniosek prezesa',
       'Ministerstwo Obrony Narodowej',
       'Przy pomocy której szef Służby Cywilnej wykonuje zadania wynikające z ustawy',
       'Na jaki okres Kierownik urzędu powołuje rzecznika dyscyplinarnego urzędu'],
      dtype=object)

The answers for f1-score higher than 0.5 consits of more complex answer that could be considered satisfying.

#### Why extractive question answering is not well suited for inflectional languages?

EQA is not well suited for inflectional languages because of word conjugation. In Polish language we can have multiple word conjugation for plural form of a word. Example: kodeksy, kodeksów etc. Also EQA relies on exact comparison of text variables because its goal is to find the exact answer within the text that best matches the query, so the word conjucagtion is a significant disadvantage there